# Week 9 Coding Quiz 

In [4]:
!pip install statsmodels 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 41.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import skew

# Reproducibility
np.random.seed(0)

print('Libraries imported. Seed set to 0.')

Libraries imported. Seed set to 0.


## Simulation function

In [11]:
def simulate(A=1, B=1, C=10, D=1000):
    """
    Simulate data for Y, X, W.
    Parameters:
        A: True coefficient on X
        B: Noise level in X (variance of U in X=W+U where U~N(0,B))
        C: Noise level in Y (std dev of the noise term)
        D: Sample size
    Returns:
        Y, X, W arrays
    """
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

Y, X, W = simulate()
len(Y), len(X), len(W)

(1000, 1000, 1000)

## Question 1
Which of the following is closest to the probability of detecting a nonzero effect of ﻿X﻿ on ﻿Y﻿ (the t-value of ﻿X﻿ is greater in absolute value than about 1.96) given A = 1, B = 1, C = 10, D = 1000? Include W in the regression.

In [7]:
def detection_probability(A=1, B=1, C=10, D=1000, reps=1000, alpha=0.05):
    crit = 1.96  # approx two-sided 5% threshold
    t_vals = []
    for _ in range(reps):
        Y, X, W = simulate(A=A, B=B, C=C, D=D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_vals.append(model.tvalues[1])  # t for X
    return np.mean(np.abs(t_vals) > crit)

prob_detect_q1 = detection_probability(A=1, B=1, C=10, D=1000, reps=1000)
print(f"Q1 — Probability of detection: {prob_detect_q1:.3f}")

Q1 — Probability of detection: 0.878


## Question 2
Which of the following is closest to the skew of the estimate in that case? (You can compute this using scipy.)


In [12]:
from scipy.stats import skew
import numpy as np
import statsmodels.api as sm

def simulate(A=1, B=1, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

coefs = []
for _ in range(1000):
    Y, X, W = simulate(A=1, B=1, C=10, D=1000)
    XW = sm.add_constant(np.column_stack([X, W]))
    model = sm.OLS(Y, XW).fit()
    coefs.append(model.params[1])  # coefficient on X

coef_skew = skew(coefs)
print("Skew of the estimate:", coef_skew)


Skew of the estimate: 0.014365290670855346


## Question 3
With A = 1, C = 10, D = 1,000, what value of B is needed to detect that the Data Generating Process (DGP) has a nonzero coefficient for X about 50% of the time? (Choose the closest value.)

In [13]:
def power_for_B(B, reps=500):
    return detection_probability(A=1, B=B, C=10, D=1000, reps=reps)

Bs = [0.2, 0.6, 1.8, 5.4]
powers_B = {B: power_for_B(B) for B in Bs}
powers_B

{0.2: np.float64(0.102),
 0.6: np.float64(0.464),
 1.8: np.float64(1.0),
 5.4: np.float64(1.0)}

## Question 4
With B = 1, C = 10, D = 100 (note the different value of D), what value of A is needed to detect that the DGP has a nonzero coefficient for X about 50% of the time? (Choose the closest value.) 

In [14]:
def power_for_A(A, reps=500):
    return detection_probability(A=A, B=1, C=10, D=100, reps=reps)

As = [0.5, 1.0, 2.0, 4.0]
powers_A = {A: power_for_A(A) for A in As}
powers_A

{0.5: np.float64(0.084),
 1.0: np.float64(0.156),
 2.0: np.float64(0.518),
 4.0: np.float64(0.986)}